In [1]:
!pip install transformers datasets gradio


In [2]:
with open("/chat_data.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

print("Total lines:", len(lines))
print("\nFirst 3 lines:")
for line in lines[:3]:
    print(line.strip())


Total lines: 48

First 3 lines:
Hello! <SEP> Hi there! How can I help you today?
What's your name? <SEP> I'm a chatbot created to assist you.
How are you? <SEP> I'm just a program, but I'm functioning as expected!


In [3]:
pairs = [line.strip().split("<SEP>") for line in lines if "<SEP>" in line]
print("Total pairs:", len(pairs))
print("Example pair:", pairs[0])


Total pairs: 48
Example pair: ['Hello! ', ' Hi there! How can I help you today?']


In [4]:
# Cell 3
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Fix for padding error: Use eos_token as pad_token
tokenizer.pad_token = tokenizer.eos_token

def tokenize_pair(pair):
    text = pair[0].strip() + " " + pair[1].strip()
    return tokenizer(text, truncation=True, padding="max_length", max_length=64)

# Tokenize all pairs
tokenized = [tokenize_pair(pair) for pair in pairs]

print("Tokenization successful!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Tokenization successful!


In [5]:
# Cell 4
import torch

class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings["input_ids"])

# Convert list of dicts to dict of lists
batch_encoding = {key: [e[key] for e in tokenized] for key in tokenized[0]}
dataset = ChatDataset(batch_encoding)

print("Dataset ready for training!")


Dataset ready for training!


In [6]:
# Cell 5
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling

model = AutoModelForCausalLM.from_pretrained("gpt2")

# Resize model embeddings for new pad_token
model.resize_token_embeddings(len(tokenizer))

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="./chatbot_model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10,
    logging_steps=5,
    learning_rate=5e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/tmp/ipython-input-6-4193668184.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: navanithas2002 (navanithas2002-k-s-rangasamy-college-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
5,3.334500
10,3.310500
15,3.337900
20,2.584100
25,2.368100
30,2.302100
35,2.009000
40,1.910100
45,2.020400
50,1.824300


TrainOutput(global_step=72, training_loss=2.2448796530564628, metrics={'train_runtime': 520.7813, 'train_samples_per_second': 0.277, 'train_steps_per_second': 0.138, 'total_flos': 4703256576000.0, 'train_loss': 2.2448796530564628, 'epoch': 3.0})

In [7]:
# Cell 6
import gradio as gr

def chat_fn(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(input_ids, max_length=60, do_sample=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)

gr.Interface(
    fn=chat_fn,
    inputs="text",
    outputs="text",
    title="My Fine-Tuned Chatbot"
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d563423ad14dfba707.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
